# Imports and helper functions

To import packages and modules to Jupyter notebook, you need to setup a conda environment. Here we call it `gpst`.
```
conda create --name gpst
conda install pypsa pandapower jupyterlab
```
Upgrade to pandapower to develop branch
```
pip install git+git://github.com:e2nIEE/pandapower@develop
```
To  add the kernel for the jupyter notebook
```
pip install ipykernel
ipython kernel install --user --name=gpst
```

Open the jupyter lab notebook by typing `jupyter lab` in the terminal.


In [1]:
import os
import timeit
import pandas as pd

import numpy as np
import pandapower as pp
import pandapower.converter
        
import logging

# Show all pandas columns in jupyter
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 8)

In [2]:
# Optional. Take local PyPSA-Dev installation to make adjustments
pypsa_path = os. getcwd()+"/PyPSA"  # require to have `PyPSA` cloned in ~/power-flow-exercise/example_pandapower/<PyPSA>`
import sys
sys.path.insert(0, f"{pypsa_path}")

In [3]:
import pypsa

In [4]:
def _sets_path_to_root(root_directory_name):
    """
    Search and sets path to the given root directory (root/path/file).

    Parameters
    ----------
    root_directory_name : str
        Name of the root directory.
    n : int
        Number of folders the function will check upwards/root directed.

    """
    import os

    repo_name = root_directory_name
    n = 8  # check max 8 levels above. Random default.
    n0 = n

    while n >= 0:
        n -= 1
        # if repo_name is current folder name, stop and set path
        if repo_name == os.path.basename(os.path.abspath(".")):
            repo_path = os.getcwd()  # os.getcwd() = current_path
            os.chdir(repo_path)  # change dir_path to repo_path
            print("This is the repository path: ", repo_path)
            print("Had to go %d folder(s) up." % (n0 - 1 - n))
            break
        # if repo_name NOT current folder name for 5 levels then stop
        if n == 0:
            print("Cant find the repo path.")
        # if repo_name NOT current folder name, go one dir higher
        else:
            upper_path = os.path.dirname(
                os.path.abspath("."))  # name of upper folder
            os.chdir(upper_path)

In [5]:
_sets_path_to_root("power-flow-exercise-gpst")  # name of the git clone folder

This is the repository path:  /home/max/OneDrive/PHD-Flexibility/07_pypsa-africa/0github/power-flow-exercise-gpst
Had to go 1 folder(s) up.


In [6]:
# Set logger
logging.basicConfig(filename=os.path.join("example_pandapower", "log.out"),
                    filemode='w',
                    format='%(asctime)s %(name)s %(levelname)s %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.INFO)

logger = logging.getLogger(__name__)

# Pandapower import of Matpower

In [7]:
net = pp.converter.from_mpc(os.path.join("reference-matpower", "RTS_GMLC.mat"))

# Comparing Pandapower to solved Matpower network

In [8]:
def compare_to_matpower(net):
    """
    Compares pandapower network to matpower network.
    """
    bus_results = pd.read_csv(os.path.join("reference-matpower", "results", "bus.csv"), index_col=0)
    branch_results = pd.read_csv(os.path.join("reference-matpower", "results", "flow.csv"), index_col=0)

    # compare bus results
    # merged_results = pd.merge(bus_results, net.res_bus, how='inner', left_index=True, right_on=net.bus.id)
    merged_results = pd.merge(bus_results, net.res_bus, how='inner', left_index=True, right_on=net.bus.name)
    merged_results['diff_vm_pu'] = merged_results.v_mag - merged_results.vm_pu
    merged_results['diff_va_degree'] = merged_results.v_ang - merged_results.va_degree
    logger.info(f"\n{merged_results[['diff_vm_pu', 'diff_va_degree']].describe()}")

    # compare branch results
    merged_results_line = pd.merge(branch_results, net.res_line, how='inner', left_index=True, right_index=True)
    merged_results_line['diff_from_p_mw'] = merged_results_line.p_from_mw - merged_results_line.from_bus_inj_p
    merged_results_line['diff_to_p_mw'] = merged_results_line.p_to_mw - merged_results_line.to_bus_inj_p
    merged_results_line['diff_from_q_mvar'] = merged_results_line.q_from_mvar - merged_results_line.from_bus_inj_q
    merged_results_line['diff_to_q_mvar'] = merged_results_line.q_to_mvar - merged_results_line.to_bus_inj_q
    merged_results_line['diff_loss_p'] = merged_results_line.pl_mw - merged_results_line.loss_p
    cols = ['diff_from_p_mw', 'diff_to_p_mw', 'diff_from_q_mvar', 'diff_to_q_mvar', 'diff_loss_p']
    logger.info(f"\n{merged_results_line[cols].describe()}")

In [9]:
net.sgen.drop(net.sgen[~net.sgen.in_service].index, inplace=True)  # TODO: check if this is correct (dropping not in_service elements from net.sgen
pp.runpp(net)
compare_to_matpower(net)  # Info: Will be saved in logger 'log.out'

# Prepare Pandapower network

In [10]:
def make_name_to_index(net):
    """
    Makes name as index.
    
    PyPSA requires a unique name as index.
    """
    for elm in net.keys():
        if elm.startswith("_") or not isinstance(net[elm], pd.DataFrame) or len(net[elm]) == 0:
            continue
        # let's keep the bus names from mpc
        if elm == 'bus':
            continue
        net[elm]['name'] = [f"{elm}_{i}" for i in net[elm].index.values]
        # checking because in_service not supported by pypsa
        if 'in_service' in net[elm].columns:
            assert np.alltrue(net[elm]['in_service']), f"not in_service elements found in {elm}"
        # checking because switches are not supported by pypsa
        if elm == 'switch':
            assert np.alltrue(net[elm]['closed']), "open switches found"
        if elm == "trafo3w" and len(net[elm]) > 0:
            logger.warning("found trafo3w in net (not supported by pypsa converter!!!)")
        if elm == "shunt" and len(net[elm]) > 0:
            logger.warning("found shunt in net (not supported by pypsa converter!!!)")

In [11]:
make_name_to_index(net)

# PyPSA import of Pandapower network

In [12]:
# Build empty PyPSA network
network = pypsa.Network()
# Import pandapower
network.import_from_pandapower_net(net, True)

### Checks if import is correct

In [13]:
# now let's check some basic infos
assert len(network.buses) == len(net.bus)
assert len(network.generators) == (len(net.gen) + len(net.sgen) + len(net.ext_grid))
assert len(network.loads) == len(net.load)
assert len(network.transformers) == len(net.trafo)
# todo: shunt impedances are not supported
# todo: trafo tap positions are not supported
# todo: trafo3w are not supported

# Inspect PyPSA network before solving

In [14]:
n = network

In [15]:
n.snapshots

Index(['now'], dtype='object', name='snapshot')

In [16]:
n.global_constraints

attribute,type,investment_period,carrier_attribute,sense,constant,mu
GlobalConstraint,,,,,,


In [17]:
n.buses

,v_nom,v_mag_pu_set,type,x,y,carrier,unit,v_mag_pu_min,v_mag_pu_max,control,sub_network
Bus,,,,,,,,,,,
100,138.0,1.0468,,0.0,0.0,AC,None,0.0,inf,PQ,
101,138.0,1.0467,,0.0,0.0,AC,None,0.0,inf,PQ,
102,138.0,1.0000,,0.0,0.0,AC,None,0.0,inf,PQ,
103,138.0,1.0000,,0.0,0.0,AC,None,0.0,inf,PQ,
...,...,...,...,...,...,...,...,...,...,...,...
321,230.0,1.0500,,0.0,0.0,AC,None,0.0,inf,PQ,
322,230.0,1.0500,,0.0,0.0,AC,None,0.0,inf,PQ,
323,230.0,1.0000,,0.0,0.0,AC,None,0.0,inf,PQ,
324,230.0,1.0000,,0.0,0.0,AC,None,0.0,inf,PQ,


In [18]:
n.generators

,p_set,q_set,bus,control,type,p_nom,p_nom_extendable,p_nom_min,p_nom_max,p_min_pu,p_max_pu,sign,carrier,marginal_cost,build_year,lifetime,capital_cost,efficiency,committable,start_up_cost,shut_down_cost,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
gen_0,-8.0,0.000000,100,PV,,0.0,False,0.0,inf,0.0,1.0,1.0,,0.0,0,inf,0.0,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
gen_1,-8.0,0.000000,101,PV,,0.0,False,0.0,inf,0.0,1.0,1.0,,0.0,0,inf,0.0,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
gen_2,-355.0,0.000000,106,PV,,0.0,False,0.0,inf,0.0,1.0,1.0,,0.0,0,inf,0.0,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
gen_3,-5.0,0.000000,114,PV,,0.0,False,0.0,inf,0.0,1.0,1.0,,0.0,0,inf,0.0,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sgen_60,-50.0,5.460065,321,PQ,,0.0,False,0.0,inf,0.0,1.0,1.0,,0.0,0,inf,0.0,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
sgen_61,-50.0,5.460065,321,PQ,,0.0,False,0.0,inf,0.0,1.0,1.0,,0.0,0,inf,0.0,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
sgen_62,-50.0,5.460065,321,PQ,,0.0,False,0.0,inf,0.0,1.0,1.0,,0.0,0,inf,0.0,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
ext_grid_0,0.0,0.000000,112,Slack,,0.0,False,0.0,inf,0.0,1.0,1.0,,0.0,0,inf,0.0,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0


In [19]:
n.loads

,p_set,q_set,bus,carrier,type,sign
Load,,,,,,
load_0,108.0,22.0,100,,,-1.0
load_1,97.0,20.0,101,,,-1.0
load_2,180.0,37.0,102,,,-1.0
load_3,74.0,15.0,103,,,-1.0
...,...,...,...,...,...,...
load_47,100.0,20.0,315,,,-1.0
load_48,333.0,68.0,317,,,-1.0
load_49,181.0,37.0,318,,,-1.0
load_50,128.0,26.0,319,,,-1.0


In [20]:
n.lines

,r,x,b,s_nom,bus0,bus1,length,num_parallel,type,g,s_nom_extendable,s_nom_min,s_nom_max,s_max_pu,capital_cost,build_year,lifetime,carrier,terrain_factor,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt
Line,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
line_0,0.57132,2.66616,0.002421,101.036297,100,101,1.0,1.0,,0.0,False,0.0,inf,1.0,0.0,0,inf,,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
line_1,10.47420,40.18284,0.000299,101.036297,100,102,1.0,1.0,,0.0,False,0.0,inf,1.0,0.0,0,inf,,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
line_2,4.18968,16.18740,0.000121,101.036297,100,104,1.0,1.0,,0.0,False,0.0,inf,1.0,0.0,0,inf,,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
line_3,6.28452,24.18588,0.000179,101.036297,101,103,1.0,1.0,,0.0,False,0.0,inf,1.0,0.0,0,inf,,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
line_101,4.76100,35.97200,0.000268,288.675135,320,321,1.0,1.0,,0.0,False,0.0,inf,1.0,0.0,0,inf,,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
line_102,6.34800,51.31300,0.000384,288.675135,324,120,1.0,1.0,,0.0,False,0.0,inf,1.0,0.0,0,inf,,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
line_103,6.87700,55.01600,0.000412,288.675135,317,222,1.0,1.0,,0.0,False,0.0,inf,1.0,0.0,0,inf,,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
line_104,0.00000,4.76100,0.000000,416.846894,322,324,1.0,1.0,,0.0,False,0.0,inf,1.0,0.0,0,inf,,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
n.transformers

,phase_shift,s_nom,bus0,bus1,r,x,g,b,tap_position,type,model,s_nom_extendable,s_nom_min,s_nom_max,s_max_pu,capital_cost,num_parallel,tap_ratio,tap_side,build_year,lifetime,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt
Transformer,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
trafo_0,0.0,0.4,123,102,0.008,0.336,0.0,-0.0,1,,t,False,0.0,inf,1.0,0.0,1.0,1.0,0,0,inf,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
trafo_1,0.0,0.4,110,108,0.008,0.336,0.0,-0.0,1,,t,False,0.0,inf,1.0,0.0,1.0,1.0,0,0,inf,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
trafo_2,0.0,0.4,111,108,0.008,0.336,0.0,-0.0,1,,t,False,0.0,inf,1.0,0.0,1.0,1.0,0,0,inf,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
trafo_3,0.0,0.4,110,109,0.008,0.336,0.0,-0.0,1,,t,False,0.0,inf,1.0,0.0,1.0,1.0,0,0,inf,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
trafo_11,0.0,0.4,310,308,0.008,0.336,0.0,-0.0,1,,t,False,0.0,inf,1.0,0.0,1.0,1.0,0,0,inf,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
trafo_12,0.0,0.4,311,308,0.008,0.336,0.0,-0.0,1,,t,False,0.0,inf,1.0,0.0,1.0,1.0,0,0,inf,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
trafo_13,0.0,0.4,310,309,0.008,0.336,0.0,-0.0,1,,t,False,0.0,inf,1.0,0.0,1.0,1.0,0,0,inf,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
trafo_14,0.0,0.4,311,309,0.008,0.336,0.0,-0.0,1,,t,False,0.0,inf,1.0,0.0,1.0,1.0,0,0,inf,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
n.links

attribute,bus0,bus1,type,carrier,efficiency,build_year,lifetime,p_nom,p_nom_extendable,p_nom_min,p_nom_max,p_set,p_min_pu,p_max_pu,capital_cost,marginal_cost,length,terrain_factor,p_nom_opt
Link,,,,,,,,,,,,,,,,,,,


In [23]:
n.stores

attribute,bus,type,carrier,e_nom,e_nom_extendable,e_nom_min,e_nom_max,e_min_pu,e_max_pu,e_initial,e_initial_per_period,e_cyclic,e_cyclic_per_period,p_set,q_set,sign,marginal_cost,capital_cost,standing_loss,build_year,lifetime,e_nom_opt
Store,,,,,,,,,,,,,,,,,,,,,,


In [24]:
n.storage_units

attribute,bus,control,type,p_nom,p_nom_extendable,p_nom_min,p_nom_max,p_min_pu,p_max_pu,p_set,q_set,sign,carrier,marginal_cost,capital_cost,build_year,lifetime,state_of_charge_initial,state_of_charge_initial_per_period,state_of_charge_set,cyclic_state_of_charge,cyclic_state_of_charge_per_period,max_hours,efficiency_store,efficiency_dispatch,standing_loss,inflow,p_nom_opt
StorageUnit,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [25]:
n.shunt_impedances

attribute,bus,g,b,sign,g_pu,b_pu
ShuntImpedance,,,,,,


In [26]:
n.carriers

attribute,co2_emissions,color,nice_name,max_growth
Carrier,,,,


# Solve network

In [27]:
#n.lopf()
#n.pf(use_seed=True)
n.pf()

{'n_iter': SubNetwork    0
 snapshot       
 now         100,
 'error': SubNetwork             0
 snapshot                
 now         5.446325e+41,
 'converged': SubNetwork      0
 snapshot         
 now         False}